In [ ]:
from SAMap import SAMAP, get_mapping_scores, GenePairFinder
from samalg import SAM

SAMap accepts file paths to unprocessed, raw `.h5ad` files. Alternatively, if you already have a processed `SAM` object, you can load them in directly.


Prior to running SAMap, you should have run `map_genes.sh`, which expects 2-character identifiers describing each species. For example, human and mouse might get `hu` and `mo` identifiers, respectively. `map_genes.sh` generates a `maps/` directory with the transcriptome mapping BLAST results deposited. The input species IDs and path to the `maps/` directory should be input into SAMap.

In [ ]:
id1 = 'hu'
id2 = 'mo'

In [ ]:
# passing in file names (SAMap will process the data with SAM and save the resulting objects to two `.h5ad` files.)
fn1 = '/path/to/file1.h5ad' #processed data will be automatically saved to `/path/to/file/file1_pr.h5ad`
fn2 = '/path/to/file2.h5ad' #processed data will be automatically saved to `/path/to/file/file2_pr.h5ad`
samap,sam1,sam2,ITER_DATA = SAMAP(fn1,fn2,id1,id2,f_maps = 'maps/')

"""
# passing in already-processed SAM objects
sam1=SAM()
sam2=SAM()
sam1.load_data('/path/to/file1_pr.h5ad')
sam2.load_data('/path/to/file2_pr.h5ad')
samap,sam1,sam2,ITER_DATA = SAMAP(sam1,sam2,id1,id2,f_maps = 'maps/')"""

To calculate alignment scores between cell types, we can use `get_mapping_scores`. This function will use the combined SAM object produced by SAMap to calculate alignment scores between ell types in the provided cell type annotation columns of `sam.adata.obs`. If no cell type annotations exist, the leiden clusters generated by SAM can be used (`k1=k2='leiden_clusters'`).

The resulting tables show the highest-scoring alignment scores for each cell type in organism 1 (`D1`) and organism 2 (`D2`), respectively.

In [ ]:
# get_mapping_scores will calculate alignment scores between cell types in the provided cell type annotation columns of `sam.adata.obs`.
# if no cell type annotations exist, the leiden clusters generated by `SAM` can be used:
# k1 = k2 = 'leiden_clusters'
k1 = 'cell_types' #cell types annotation key in `sam1.adata.obs`
k2 = 'cell_types' #cell types annotation key in `sam2.adata.obs`
D1,D2 = get_mapping_scores(sam1,sam2,samap,k1,k2)

SAMap provides a class to find gene pairs enriched in different cell type pairs. The method entails finding gene pairs that contribute positively to the cross-species correlation between cell types and are differentially expressed in their respective mapped cell types.

In [ ]:
gpf = GenePairFinder(sam1,sam2,samap,k1=k1,k2=k2)

`gpf.find_genes` can now be used to find gene pairs enriched in a cell type mapping.

In [ ]:
n1 = 'cell_type1' #cell type ID from organism 1 (must be present in `sam1.adata.obs[k1]`)
n2 = 'cell_type2' #cell type ID from organism 2 (must be present in `sam2.adata.obs[k1]`)
Gp,G1,G2 = gpf.find_genes(n1,n2)
#Gp are the gene pairs, G1 are the genes from organism 1, G2 are the genes from organism 2